#### Kaggle 대회 랭킹 1위인 andrey lukyanenko의 notebook을 참고하여 정리한 내용입니다.

## General information

Don't Overfit II 에서는 binary classification을 합니다.

train 샘플은 300열 250행의 크기이고, test 샘플은 train 샘플의 크기의 79배입니다.	

1)train data의 숫자가 test data의 수 보다 압도적으로 적은데다가, 아래에 EDA 결과가 나옵니다만, 2)target class data의 불균형으로 인해 과적합의 가능성이 매우 높습니다!	

본 competition의 목표는 오버피팅하지 않는 모델을 만드는 것입니다!!

작성한 notebook에서는 아래와 같은 작업들을 수행합니다!

* 인사이트를 얻기위해 항목에 대한 EDA를 합니다
* permutation importance를 사용하여 가장 영향력 있는 항목을 찾습니다	
* 여러 모델을 비교합니다 - bayes classification, 선형 모델, 트리기반 모델 등을 해봅니다
* 여러 종류의 feature selection 방법을 해봅니다 - ELI5 및 SHAP을 포함합니다 (+ feed forward, backward elimination, stepwise 방법도 고려할 것!)
* hyper parameter 최적화를 해봅니다
* feature engineering(몇 개의 feature를 추가해봄)

![](https://cdn-images-1.medium.com/max/1600/1*vuZxFMi5fODz2OEcpG-S1g.png)

In [ ]:
# Libraries
import numpy as np
import pandas as pd
from scipy import stats
pd.set_option('max_columns', None)
import json
import ast  # abstract syntax tree
import time
import datetime
import os
from operator import itemgetter  # operator : 파이썬 고유 연산자에 대해 효율적인 함수 세트를 제공함, itemgetter : 복잡한 소스 및 객체에서 특정 값을 추출함.
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.offline as py  # offline으로 그래프를 만들어 local에 저장
py.init_notebook_mode(connected=True)  # plotly 시작
import plotly.graph_objs as go  # 속성 값에 대한 유효성 제공
import plotly.tools as tls  # 플롯 모듈을 가져옴
# 정우일님의 관련 블로그 https://wooiljeong.github.io/python/python_plotly/

from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
# StratifiedKFold : KFold의 변형의 한 종류
# RepeatedStratifiedKFold : 임의로 계층화 된(shuffling) K-Fold를 N번 반복
# cross_val_score : 교차 검증 점수를 평가
# accuracy_score : 교차 검증에서 정확도를 기록
# f_classif : 분산 분석(ANOVA, 3개 이상 그룹에 대하여 평균의 동등성을 통계적으로 계산)의 F-value를 계산
# mutual_info_classif : 이산 목표 변수에 대한 상호 정보를 추정
# RFECV : 재귀적 feature elimination + top-N score에 해당하는 feature들에 대해 교차 검증을 실시
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import GenericUnivariateSelect, SelectPercentile, SelectKBest, f_classif, mutual_info_classif, RFECV


import xgboost as xgb
import lightgbm as lgb  # Gradient boosting의 변형 기법 중 하나. 다른 알고리즘과 달리, tree를 수직으로 확장하여 과적합 현상을 방지
from sklearn.svm import SVC
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import AdaBoostClassifier

from sklearn import linear_model
import statsmodels.api as sm

# ELI5 : 머신러닝 분류기를 디버그하고 예측을 설명하는데 도움을 줌
# PermutationImportance : 블랙박스 예측기의 feature importance를 계산
# SHAP : 모든 머신러닝 모델의 출력을 설명하는 게임 이론적 접근 방식 
import eli5  
from eli5.sklearn import PermutationImportance
import shap

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = '/kaggle/input/dont-overfit-ii'
train = pd.read_csv(f'{path}/train.csv')
test = pd.read_csv(f'{path}/test.csv')
train.shape

In [ ]:
test.shape

구글 콜랩에서 사용하실 때는 컴퓨터에 첨부된 트레인 및 테스트 csv 파일을 컴퓨터에 다운로드 한 후 아래 코드를 실행하여 다시 그 파일들을 불러올 수 있게 됩니다.

    from google.colab import files
    uploaded = files.upload()

그런 다음 아래 코드를 통해서 csv를 데이터프레임으로 바꿀 수 있게 됩니다.

    import io
    test = pd.read_csv(io.BytesIO(uploaded['test.csv']))
    train = pd.read_csv(io.BytesIO(uploaded['train.csv']))

<a id="de"></a>
## Data exploration

In [ ]:
train.head()

* id 열, target 열 및 300개의 feature가 있음을 알 수 있습니다
* 익명으로 처리되었으므로 그 의미를 모릅니다
* feature 간의 관계를 최대한 이해해보도록 하겠습니다.

In [ ]:
train[train.columns[2:]].std().plot('hist');
plt.title('Distribution of stds of all columns');

In [ ]:
train[train.columns[2:]].mean().plot('hist');
plt.title('Distribution of means of all columns');

* 모든 열의 평균 값이 -0.2와 0.15 사이임을 알 수 있습니다	
* 표준 편차는 매우 작습니다
* 우리는 항목이 서로 매우 비슷하다고 말할 수 있습니다

In [ ]:
# we have no missing values
train.isnull().any().any()

In [ ]:
print('Distributions of first 28 columns')
plt.figure(figsize=(26, 24))
for i, col in enumerate(list(train.columns)[2:30]):
    plt.subplot(7, 4, i + 1)
    plt.hist(train[col])
    plt.title(col)

In [ ]:
train['target'].value_counts()

이 개요에서 우리는 다음을 볼 수 있습니다

* 대상은 이진이며 약간의 불균형이 있습니다 -> 성능 측정 시, 단순히 accuracy만 고려하기에는 무리가 있음
* 샘플의 26.8 %가 0 클래스에 속합니다
* 열의 값은 다소 비슷합니다

이제 상관 관계를 살펴 봅시다!

In [ ]:
corr = train[train.columns[2:]].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(25, 25))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5)

열이 너무 많아서 위에서 도저히 읽을 수가 없습니다.

top correlated features를 보겠습니다.

In [ ]:
corrs = train.corr().abs().unstack().sort_values(kind="quicksort").reset_index()
corrs = corrs[corrs['level_0'] != corrs['level_1']]
corrs.tail(10)

feature 간의 상관 관계가 0.3보다 낮고 target과 가장 관련이 높은 feature의 상관 관계는 0.33입니다	

따라서 제거 할 수 있는 상관 관계가 높은 feature가 없으며, target과 상관 관계가 거의 없는 일부 feature를 삭제할 수 있습니다.

## Prepare the data

In [ ]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']
X_test = test.drop(['id'], axis=1)

## Basic modelling

기본 모델링을합니다

train data가 충분하지 않기 때문에, 과적합의 가능성이 높아집니다.

이를 방지하기 위해, model은 기본적으로 K-Fold Cross Validation 방식을 채택하겠습니다.

이 떄, 성능 측정을 위해 sklearn의 cross_val_score 대신, roc_auc_score를 사용하도록 하겠습니다.

왜냐하면 cross_val_score는,	

* 예측값을 제공하지 않는다.
* 폴드로부터의 예측은 제공하지 않는다
* 특정 변환을 적용할 수 없다. (EX : 특정 feature selector의 input formatting을 위한 변환 작업)
* lgbm, catboost, xgboost와 같은 그래디언트 부스팅 모델은 cross_val_score로 전달할 수 없는 추가 매개 변수를 필요로한다.	

로직은 아래와 같습니다.

```
for fold in folds:
    get train and validation data
    apply some transformations (if necessary)
    train model
    predict on validation data
    calculate train and validation metrics(rou 커브의 auc score)
    predict on test data
```

간단한 logistic regression을 사용하여 단계별로 코드를 작성할 것입니다.

[](http://i.imgur.com/QBuDOjs.jpg)

먼저 몇 가지 사항을 정의 해 보겠습니다	

* Prediction은 우리의 예측이 될 것입니다
* scores_train, scores_valid은 점수 리스트입니다
* fold란 데이터를 나누는 방법입니다

In [ ]:
prediction = np.zeros(len(X_test))  # 예측 값(각 클래스의 출현 확률)을 저장할 배열
scores_train = []  # train roc_auc_score
scores_valid = []  # validation roc_auc_score
folds = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)

이제 폴드를 한 번 사용하여 트레인 데이터를 훈련 및 검증으로 분할합니다

In [ ]:
for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train, y_train)):
    X_train_fold, X_valid_fold = X_train.loc[train_index], X_train.loc[valid_index]
    y_train_fold, y_valid_fold = y_train[train_index], y_train[valid_index]
    break

이제 모델을 훈련시키고 roc 커브의 auc score를 계산할 수 있습니다.

In [ ]:
# C값이 작을수록, 정규화의 강도가 커짐. 또한 feature가 매우 많은 경우, 영향력이 없는(계수가 0에 가까운) 변수를 제거하는 lasso 방식을 사용하는 것이 좋음.
model = linear_model.LogisticRegression(class_weight='balanced', penalty='l1', C=0.1, solver='liblinear')
model.fit(X_train_fold, y_train_fold)
y_pred_train = model.predict(X_train_fold).reshape(-1,)  # train set 예측
train_score = roc_auc_score(y_train_fold, y_pred_train)  # sensitivity(recall), 1-specificity 두 개의 축을 활용해, ROC 커브의 AUC 값 계산

y_pred_valid = model.predict(X_valid_fold).reshape(-1,)  # validation set 예측
valid_score = roc_auc_score(y_valid_fold, y_pred_valid)  

# roc_auc_score : https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc?hl=ko
# sensitivity(민감도 or 재현율), specificity(특이도) : https://blog.naver.com/PostView.nhn?blogId=jesus24968&logNo=220734872380

In [ ]:
print(f'Train auc: {train_score:.4}. Valid auc: {valid_score:.4}')

위에서 설명한 로직을 토대로 함수를 만듭니다.

In [ ]:
def train_model(X_train, y_train, X_test, folds=folds, model=None):
    prediction = np.zeros(shape = (len(X_test), 2))
    scores_train = []
    scores_valid = []
    
    # 지정된 fold 수 만큼 model training / train&valid set prediction / test set prediction을 수행 
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train, y_train)):
        X_train_fold, X_valid_fold = X_train[train_index], X_train[valid_index]
        y_train_fold, y_valid_fold = y_train[train_index], y_train[valid_index]
        
        model.fit(X_train_fold, y_train_fold)
        
        y_pred_train = model.predict(X_train_fold).reshape(-1,)
        train_score = roc_auc_score(y_train_fold, y_pred_train)
        scores_train.append(train_score)
        
        y_pred_valid = model.predict(X_valid_fold).reshape(-1,)
        valid_score = roc_auc_score(y_valid_fold, y_pred_valid)
        scores_valid.append(valid_score)

        y_pred = model.predict_proba(X_test)  # 각 class(0 또는 1)의 출현 확률을 계산
        prediction += y_pred

    prediction /= folds.get_n_splits()
    prediction = np.argmax(prediction, axis = 1) # 확률 값이 높은 인덱스를 선택(class)
    
    print(f'Mean train auc: {np.mean(scores_train):.4f}, std: {np.std(scores_train):.4f}.')
    print(f'Mean valid auc: {np.mean(scores_valid):.4f}, std: {np.std(scores_valid):.4f}.')
    
    return scores_valid, prediction


기본 모델을 바탕으로 Logistic Regression 기법을 적용한 결과, 성능은 아래와 같습니다.

In [ ]:
model = linear_model.LogisticRegression(class_weight='balanced', penalty='l1', C=0.1, solver='liblinear')
scores, prediction = train_model(X_train.values, y_train, X_test, folds=folds,  model=model)

### Different ways of splitting data into folds

데이터를 폴드로 나누는 방법에는 여러 가지가 있습니다
* 가장 간단한 방법은 무작위로 나누는 것입니다: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
* 일반적으로 분류에는 더 나은 방법이 있긴 합니다 - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html 

StratifiedKFold는 계층화 된 폴드를 반환하는 k-폴드의 변형입니다. (주로 StratifiedKFold는 분류 문제에서, KFold는 회귀 문제에서 사용)

각 세트에는 각 세트의 샘플이 전체 세트와 거의 같은 비율로 포함되어 있습니다 (class biased한 problem을 최대한 해결하려 노력)

* StratifiedKFold와 비슷한 RepeatedStratifiedKFold도 있는데 https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html 이는 그 안에서 여러번 반복됩니다

우리는 RepeatedStratifiedKFold로 확인하겠습니다

In [ ]:
repeated_folds = RepeatedStratifiedKFold(n_splits=20, n_repeats=5, random_state=42)

In [ ]:
model = linear_model.LogisticRegression(class_weight='balanced', penalty='l1', C=0.1, solver='liblinear')
scores, prediction = train_model(X_train.values, y_train, X_test, folds=repeated_folds, model=model)

1. mean auc가 증가한 것을 볼 수 있습니다 (KFold : 0.6871 -> RepeatedStratifiedKFold : 0.7124)

## Approaches to feature selection

feature selection이 무엇이고 왜 중요한지 설명하겠습니다


### ELI5

ELI5는 ML 모델에 대한 설명을 제공하는 패키지입니다. 통합 패키지를 통해, 다양한 ML Model을 시각화할 수 있으며, 디버깅도 가능합니다.

선형 모델뿐만 아니라 트리 기반 알고리즘에 대해서도 이를 수행 할 수 있습니다.

여러 ML 프레임워크를 지원하며, 블랙 박스 모델(input에 따른 output을 제공하나, 내부 프로세스는 파악할 수 없는 모델)을 설명하는 방법을 제공합니다. 

ex) Tree 기반 모델 : 트리의 깊이가 깊어질수록, feature의 수가 많아질수록, 어떠한 프로세스에 따라 tree의 분기를 실시하였는지 설명하기 힘들어진다.
                    importance가 높은 feature에 대해 값을 바꾸면서 전체 모델 성능 변화를 확인하거나, imformation gain, 지니 계수 값의 변화를 확인하는 방식으로 설명하는 방법이 있으나,
                    이 또한 한계점이 있다.

모델의 매개 변수를 파악하고, 모델이 전체적으로 어떻게 동작하는지 확인.

In [ ]:
eli5.show_weights(model, top=50)

In [ ]:
(model.coef_ != 0).sum()

가중치가 매우 높은 항목과 가중치가 마이너스인 더 많은 항목이 있음을 알 수 있습니다	

실제로 ELI5에 따르면 중요한 항목은 32개만 있습니다	

이 항목들만 사용하여 모델을 구축해 봅시다

In [ ]:
eli5.formatters.as_dataframe.explain_weights_df(model).feature

In [ ]:
top_features = [i[1:] for i in eli5.formatters.as_dataframe.explain_weights_df(model).feature if 'BIAS' not in i]

In [ ]:
top_features

1. Important information about ELI5:

실제로 매우 간단하게 작동됩니다	
logistic regression와 같은 모델의 model coefficient를 보여주거나 랜덤 포레스트와 같은 모델의 feature importance를 보여 줍니다	

ELI5의 결과를 model coefficient와 비교해 봅니다

In [ ]:
for i, coef in enumerate(model.coef_[0]):
    if coef != 0:
        print(f'Feature {X_train.columns[i]} has coefficient {coef:.4f}')

여기에 중요한 결론이 있습니다

모델에 계수(Coefficient) 또는 항목 중요도(feature importance)가 없는 경우 ELI5가 작동하지 않습니다	

SVC가 그런 예입니다	

In [ ]:
X_train_selected = train[top_features]
y_train = train['target']
X_test_selected = test[top_features]

In [ ]:
model = linear_model.LogisticRegression(class_weight='balanced', penalty='l1', C=0.1, solver='liblinear')
scores, prediction = train_model(X_train_selected.values, y_train, X_test_selected, folds=repeated_folds, model=model)

모델이 훨씬 좋아졌다는 것을 알 수 있습니다 (Mean valid auc : before feature selection 0.7124 -> after 0.7526)

<a id="eli5p"></a>
### Permutation importance

ELI5를 잘 이용하는 다른 방법이 하나 더 있습니다

Permutation Feature Importance는 데이터가 테이블 형식일 때 훈련된 estimator에 사용할 수 있는 모델 검사 기술입니다	 

Permutation Importance는 다음과 같은 방식으로 작동합니다	

* 모델을 train한다.
* 하나의 유효성 검사 데이터 열을 지정하여, 행 데이터를 임의로 shuffling하고 점수를 계산한다.
* 점수가 크게 떨어지면 항목이 중요하다는 의미이다.

Permutation importance 참고 링크 : https://www.kaggle.com/dansbecker/permutation-importance

In [ ]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']
X_test = test.drop(['id'], axis=1)
model = linear_model.LogisticRegression(class_weight='balanced', penalty='l1', C=0.1, solver='liblinear')
scores, prediction = train_model(X_train.values, y_train, X_test, folds=repeated_folds,  model=model)

In [ ]:
submission = pd.read_csv(f'{path}/sample_submission.csv')
submission['target'] = prediction
submission.to_csv('submission_3.csv', index=False)

In [ ]:
# permutation importance
perm = PermutationImportance(model, random_state=1).fit(X_train, y_train)
eli5.show_weights(perm, top=50)

In [ ]:
eli5.formatters.as_dataframe.explain_weights_df(perm).head()

In [ ]:
eli5.formatters.as_dataframe.explain_weights_df(perm).loc[eli5.formatters.as_dataframe.explain_weights_df(perm)['weight'] != 0].shape

실제로 모델에 영향을 끼치는 feature의 수는 32개 정도라는 것을 파악할 수 있습니다.

In [ ]:
selected_weights = eli5.formatters.as_dataframe.explain_weights_df(perm).loc[eli5.formatters.as_dataframe.explain_weights_df(perm)['weight'] != 0]

In [ ]:
top_features = [i[1:] for i in selected_weights.feature if 'BIAS' not in i]
X_train_selected = train[top_features]
y_train = train['target']
X_test_selected = test[top_features]

In [ ]:
model = linear_model.LogisticRegression(class_weight='balanced', penalty='l1', C=0.1, solver='liblinear')
scores, prediction = train_model(X_train_selected.values, y_train, X_test_selected, folds=repeated_folds, model=model)

모델이 훨씬 좋아졌다는 것을 알 수 있습니다 (Mean valid auc : before feature selection 0.7124 -> after 0.7526)


### SHAP

또 다른 흥미로운 도구는 SHAP입니다	

다양한 모델에 대한 설명을 제공합니다

In [ ]:
model = linear_model.LogisticRegression(class_weight='balanced', penalty='l1', C=0.1, solver='liblinear')
scores, prediction = train_model(X_train.values, y_train, X_test, folds=repeated_folds, model=model)

In [ ]:
explainer = shap.LinearExplainer(model, X_train)
shap_values = explainer.shap_values(X_train)

shap.summary_plot(shap_values, X_train)


항목이 예측에 미치는 영향을 보여줍니다. 각 행은 각 feature을 나타냅니다.	

색상은 실제 항목 값입니다	

예를 들어 feature 18의 파란색 낮은 값은 모형 예측에 부정적인 영향을 미칩니다 (1이냐 0이냐에서 0이 되겠지요)	

빨간색 높은 값은 긍정적인 영향을 미칩니다 (1이냐 0이냐에서 1이 되겠지요)

feature 176은 반대 영향이 있습니다 	

낮은 값은 긍정적인 영향을 미치며 높은 값은 부정적인 영향을 미칩니다	

시각화를 하여, 각 feature의 모델에 대한 기여도를 쉽게 파악할 수 있다는 장점이 있습니다.

하지만, 결국 우리가 사용해야 할 feature을 수동으로 선택해야한다는 단점이 존재합니다. 	

따라서, 아래에서 그 작업을 해주는 라이브러리를 사용하겠습니다.(RFECV)

### Recursive feature elimination



In [ ]:
def train_model_with_feature_selection(X_train, y_train, X_test, folds=folds, model=None, feature_selector=None):
    prediction = np.zeros(shape = (len(X_test), 2))
    scores_train = []
    scores_valid = []
    
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train, y_train)):
        # 1. set train data & test data (fold)
        X_train_fold, X_valid_fold = X_train[train_index], X_train[valid_index]
        y_train_fold, y_valid_fold = y_train[train_index], y_train[valid_index]
        # so that we don't transform the original test data
        X_test_copy = X_test.copy()
        
        # 2. fit feature selector and transform folded data for feature selecting
        feature_selector.fit(X_train_fold, y_train_fold)
        X_train_fold = feature_selector.transform(X_train_fold)
        X_valid_fold = feature_selector.transform(X_valid_fold)
        X_test_copy = feature_selector.transform(X_test_copy)
        
        # 3. train model 
        model.fit(X_train_fold, y_train_fold)
        
        # 4. predict train and validation data
        y_pred_train = model.predict(X_train_fold).reshape(-1,)        
        y_pred_valid = model.predict(X_valid_fold).reshape(-1,)
        
        # 5. calculate roc-auc score 
        train_score = roc_auc_score(y_train_fold, y_pred_train)
        valid_score = roc_auc_score(y_valid_fold, y_pred_valid)                 
        scores_train.append(train_score)
        scores_valid.append(valid_score)

        # 6. predict test data by using predict_proba function
#         y_pred = model.predict_proba(X_test_copy)[:, 1]
        y_pred = model.predict_proba(X_test_copy)
        prediction += y_pred

    prediction /= folds.get_n_splits()
    prediction = np.argmax(prediction, axis = 1)
    
    print(f'Mean train auc: {np.mean(scores_train):.4f}, std: {np.std(scores_train):.4f}.')
    print(f'Mean valid auc: {np.mean(scores_valid):.4f}, std: {np.std(scores_valid):.4f}.')
    
    return scores_valid, prediction


우리 버전의 수정된 버전을 작성해 봅니다	

여기 교차 교차 데이터 내에 RFECV를 추가합니다

In [ ]:
# define logistic regression model
model = linear_model.LogisticRegression(class_weight='balanced', penalty='l1', C=0.1, solver='liblinear')
# define RFECV model
# 1. min_features_to_select : 선택할 feature 수의 최솟값
# 2. step : 각 단계마다, 제거할 feature의 갯수(양의 정수) 또는 비율(0~1.0 사이 값)
# 3. cv : fold model (여기서는 repeated_folds = RepeatedStratifiedKFold(n_splits=20, n_repeats=5, random_state=42) 사용)
feature_selector = RFECV(model, min_features_to_select=10, scoring='roc_auc', step=0.1, verbose=0, cv=repeated_folds, n_jobs=-1)
scores, prediction = train_model_with_feature_selection(X_train.values, y_train, X_test, folds=repeated_folds, model=model, feature_selector=feature_selector)

오히려, feature selection 이전 모델의 validation score가 조금 더 높은 모습을 확인할 수 있음.

Mean valid auc: 0.7124, std: 0.1360. -> Mean valid auc: 0.7071, std: 0.1405.

## Comparing models

다른 모델을 비교할 수 있습니다 

기본 매개 변수가있는 모델이 제대로 작동하지 않을 수 있으므로 최적화 된 모델을 비교할 가치가 있다고 생각합니다	

다음과 같이 할 것입니다:

* default parameter로 모델을 학습하고 기본 점수를 확인합니다	 
* best feature들을 선택합니다	 
* grid search를 실행합니다 
* best model을 훈련시키고 다시 점수를 봅니다	

또한 각 모델에 대한 feature selection을 해봅니다  

그리고 훈련을 더 빠르게 하기 위해 반복하지 않는 간단한 폴드를 사용할 것입니다

In [ ]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']

1. Logistic Regression

In [ ]:
model = linear_model.LogisticRegression(class_weight='balanced', penalty='l1', C=0.1, solver='liblinear')
print('Default scores')
scores, prediction = train_model(X_train.values, y_train, X_test, folds=folds, model=model)
print()
top_features = [i[1:] for i in eli5.formatters.as_dataframe.explain_weights_df(model).feature if 'BIAS' not in i]
X_train_selected = train[top_features]
y_train = train['target']
X_test_selected = test[top_features]

lr = linear_model.LogisticRegression(max_iter=1000)

parameter_grid = {'class_weight' : ['balanced', None],
                  'penalty' : ['l2', 'l1'],
                  'C' : [0.001, 0.05, 0.08, 0.01, 0.1, 1.0, 10.0],
                  'solver': ['liblinear']
                 }

grid_search = GridSearchCV(lr, param_grid=parameter_grid, cv=folds, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train_selected, y_train)
print(f'Best score of GridSearchCV: {grid_search.best_score_}')
print(f'Best parameters: {grid_search.best_params_}')

print()
scores_logreg, prediction = train_model(X_train_selected.values, y_train, X_test_selected, folds=repeated_folds, model=grid_search.best_estimator_)

모델을 최적화하면 실제로 auc 점수가 향상됩니다!
(valid 기준 auc : 0.6871 -> 0.8515)

2. AdaBoost Classifier

In [ ]:
model = AdaBoostClassifier()
print('Default scores')
scores, prediction = train_model(X_train.values, y_train, X_test, folds=folds, model=model)
print()
top_features = [i[1:] for i in eli5.formatters.as_dataframe.explain_weights_df(model).feature if 'BIAS' not in i]
X_train_selected = train[top_features]
y_train = train['target']
X_test_selected = test[top_features]


abc = AdaBoostClassifier()

parameter_grid = {'n_estimators': [5, 10, 20, 50, 100],
                  'learning_rate': [0.001, 0.01, 0.1, 1.0, 10.0]
                 }

grid_search = GridSearchCV(abc, param_grid=parameter_grid, cv=folds, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train_selected, y_train)
print(f'Best score of GridSearchCV: {grid_search.best_score_}')
print(f'Best parameters: {grid_search.best_params_}')

print()
scores_abc, prediction = train_model(X_train_selected.values, y_train, X_test_selected, folds=repeated_folds, model=grid_search.best_estimator_)

3. SGD Classifier

In [ ]:
model = linear_model.SGDClassifier(eta0=1, max_iter=1000, tol=0.0001, loss='modified_huber')
print('Default scores')
scores, prediction = train_model(X_train.values, y_train, X_test, folds=folds, model=model)
print()
top_features = [i[1:] for i in eli5.formatters.as_dataframe.explain_weights_df(model).feature if 'BIAS' not in i]
X_train_selected = train[top_features]
y_train = train['target']
X_test_selected = test[top_features]

sgd = linear_model.SGDClassifier(eta0=1, max_iter=1000, tol=0.0001)

parameter_grid = {'loss': ['log', 'modified_huber'],
                  'penalty': ['l1', 'l2', 'elasticnet'],
                  'alpha': [0.001, 0.01, 0.1, 0.5],
                  'l1_ratio': [0, 0.15, 0.5, 1.0],
                  'learning_rate': ['optimal', 'invscaling', 'adaptive']
                 }

grid_search = GridSearchCV(sgd, param_grid=parameter_grid, cv=folds, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train_selected, y_train)
print(f'Best score of GridSearchCV: {grid_search.best_score_}')
print(f'Best parameters: {grid_search.best_params_}')

print()
scores_sgd, prediction = train_model(X_train_selected.values, y_train, X_test_selected, folds=repeated_folds, model=grid_search.best_estimator_)

계수나 항목 중요도 등이 없기 때문에 SVC에서는 Permutation Importance를 사용합니다

4. Support Vector Machine

In [ ]:
model = SVC(probability=True, gamma='scale')
print('Default scores')
scores, prediction = train_model(X_train.values, y_train, X_test, folds=folds, model=model)
print()
perm = PermutationImportance(model, random_state=1).fit(X_train, y_train)
selected_weights = eli5.formatters.as_dataframe.explain_weights_df(perm).loc[eli5.formatters.as_dataframe.explain_weights_df(perm)['weight'] != 0]
top_features = [i[1:] for i in selected_weights.feature if 'BIAS' not in i]
X_train_selected = train[top_features]
y_train = train['target']
X_test_selected = test[top_features]

svc = SVC(probability=True, gamma='scale')

parameter_grid = {'C': [0.01, 0.1, 1.0, 10.0, 100.0],
                  'kernel': ['linear', 'poly', 'rbf'],
                 }

grid_search = GridSearchCV(svc, param_grid=parameter_grid, cv=folds, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train_selected, y_train)
print(f'Best score of GridSearchCV: {grid_search.best_score_}')
print(f'Best parameters: {grid_search.best_params_}')

print()
scores_svc, prediction = train_model(X_train_selected.values, y_train, X_test_selected, folds=repeated_folds, model=grid_search.best_estimator_)

In [ ]:
plt.figure(figsize=(12, 8));
scores_df = pd.DataFrame({'LogisticRegression': scores_logreg})
scores_df['AdaBoostClassifier'] = scores_abc
scores_df['SGDClassifier'] = scores_sgd
scores_df['SVC'] = scores_svc

sns.boxplot(data=scores_df);
plt.xticks(rotation=45);

logistic regression가 대부분의 다른 모델보다 우수하다는 것을 알 수 있습니다 

다른 모델은이 작은 데이터 세트에서 과적합하거나 작동하지 않는 것 같습니다 

## Feature engineering

항목 생성에는 여러 접근 방식이 있습니다	

익명화되고 비슷한 항목들이 있으면 행을 기준으로 항목을 계산할 수 있습니다

예를 들어 행 별 평균값 같은 것을 말합니다

In [ ]:
X_train['mean'] = X_train.mean(axis=1)
X_train['kurt'] = X_train.kurt(axis=1)
X_train['mad'] = X_train.mad(axis=1)
X_train['median'] = X_train.median(axis=1)
X_train['max'] = X_train.max(axis=1)
X_train['min'] = X_train.min(axis=1)
X_train['skew'] = X_train.skew(axis=1)
X_train['sem'] = X_train.sem(axis=1)

X_test['mean'] = X_test.mean(axis=1)
X_test['kurt'] = X_test.kurt(axis=1)
X_test['mad'] = X_test.mad(axis=1)
X_test['median'] = X_test.median(axis=1)
X_test['max'] = X_test.max(axis=1)
X_test['min'] = X_test.min(axis=1)
X_test['skew'] = X_test.skew(axis=1)
X_test['sem'] = X_test.sem(axis=1)

In [ ]:
model = linear_model.LogisticRegression(class_weight='balanced', penalty='l1', C=0.1, solver='liblinear')
print('Default scores')
scores, prediction = train_model(X_train.values, y_train, X_test, folds=folds, model=model)
print()
top_features = itemgetter([int(i[1:]) for i in eli5.formatters.as_dataframe.explain_weights_df(model).feature if 'BIAS' not in i])(X_train.columns)
X_train_selected = X_train[top_features]
y_train = train['target']
X_test_selected = X_test[top_features]

lr = linear_model.LogisticRegression(max_iter=1000)

parameter_grid = {'class_weight' : ['balanced', None],
                  'penalty' : ['l2', 'l1'],
                  'C' : [0.001, 0.05, 0.08, 0.01, 0.1, 1.0, 10.0],
                  'solver': ['liblinear']
                 }

grid_search = GridSearchCV(lr, param_grid=parameter_grid, cv=folds, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train_selected, y_train)
print(f'Best score of GridSearchCV: {grid_search.best_score_}')
print(f'Best parameters: {grid_search.best_params_}')

print()
scores_logreg, prediction = train_model(X_train_selected.values, y_train, X_test_selected, folds=repeated_folds, model=grid_search.best_estimator_)

CV 점수가 feature engineering 이전 모델에 비해 0.04 정도 낮아진 것을 확인할 수 있습니다. -> 이번 case의 경우 효과가 없는걸로..

## Scaling the data

과적합을 방지하기 위한 마지막 방법은 데이터의 값을 정규화하는 것입니다

일반적으로 다음과 같은 접근 방식이 있습니다

* 각 폴드에서 데이터를 트레인과 검증용으로 나눈 다음 
* 트레인데이터 및 검증데이터에 스케일러를 적용한 후 
* 다시 검증 및 테스트에 적용하는 것입니다	
* 앞처럼 다시 진행해 봅니다	

그러나 Kaggle에서는 테스트 데이터를 즉시 적용할 수 있는 독특한 상황입니다	

따라서 사용 가능한 모든 데이터에 스케일러를 적용해보기도 합니다	

데이터를 다시 준비하고 stanard scaler를 사용합니다

In [ ]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']
X_test = test.drop(['id'], axis=1)

In [ ]:
sc = StandardScaler()
data = StandardScaler().fit_transform(np.concatenate((X_train, X_test), axis=0))
X_train.iloc[:, :] = data[:250]
X_test.iloc[:, :] = data[250:]

In [ ]:
model = linear_model.LogisticRegression(class_weight='balanced', penalty='l1', C=0.1, solver='liblinear')
print('Default scores')
scores, prediction = train_model(X_train.values, y_train, X_test, folds=folds, model=model)
print()
top_features = itemgetter([int(i[1:]) for i in eli5.formatters.as_dataframe.explain_weights_df(model).feature if 'BIAS' not in i])(X_train.columns)
X_train_selected = X_train[top_features]
y_train = train['target']
X_test_selected = X_test[top_features]

lr = linear_model.LogisticRegression(max_iter=1000)

parameter_grid = {'class_weight' : ['balanced', None],
                  'penalty' : ['l2', 'l1'],
                  'C' : [0.001, 0.05, 0.08, 0.01, 0.1, 1.0, 10.0],
                  'solver': ['liblinear']
                 }

grid_search = GridSearchCV(lr, param_grid=parameter_grid, cv=folds, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train_selected, y_train)
print(f'Best score of GridSearchCV: {grid_search.best_score_}')
print(f'Best parameters: {grid_search.best_params_}')

print()
scores_logreg, prediction = train_model(X_train_selected.values, y_train, X_test_selected, folds=repeated_folds, model=grid_search.best_estimator_)

StandardScaler()를 통해 데이터를 정규화한 결과, validation auc 값이 소폭 상승한 것을 확인할 수 있습니다.
0.8515 -> 0.8538

In [ ]:
prediction

In [ ]:
# version commit 없이 결과 csv 파일을 download
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link( df, title = "Download CSV file", filename = "submission.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

submission = pd.read_csv(f'{path}/sample_submission.csv')
submission['target'] = prediction
create_download_link(submission)

In [ ]:
# submission = pd.read_csv(f'{path}/sample_submission.csv')
# submission['target'] = prediction
# submission.to_csv('submission.csv', index=False)

점수가 조금 증가했습니다!